# patch generation 
편집자: MAI&CV lab 소속 오예인 (yeinoh36@gmail.com)

In [1]:
import os
import cv2
import openslide
import numpy as np
from skimage import filters
from lxml import etree
from tqdm import trange

In [2]:
def get_ratio_mask(patch):
    h_, w_ = patch.shape[0], patch.shape[1]
    n_total = h_*w_
    n_cell = np.count_nonzero(patch)
    if (n_cell != 0):
        return n_cell*1.0/n_total*1.0
    else:
        return 0
    
def save_image(save_path, patch_name, image_patch):
    os.makedirs(save_path, exist_ok = True)
    cv2.imwrite(os.path.join(save_path, patch_name), image_patch)

def execute_patch(image_patch, target_patch_size, save_path, start_levelm_x, start_levelm_y, patch_count, patch_label):
    resize_image = cv2.resize(image_patch, (target_patch_size,target_patch_size), cv2.INTER_AREA)
    save_image(save_path, f'{start_levelm_x}_{start_levelm_y}_{patch_count}_{patch_label}.png', resize_image)

def get_tissue_mask(rgb_image):
    hsv = cv2.cvtColor(rgb_image,cv2.COLOR_RGB2HSV)
    tissue_S = hsv[:, :, 1] > filters.threshold_otsu(hsv[:, :, 1])
    background_R = rgb_image[:, :, 0] > filters.threshold_otsu(rgb_image[:, :, 0])
    background_G = rgb_image[:, :, 1] > filters.threshold_otsu(rgb_image[:, :, 1])
    background_B = rgb_image[:, :, 2] > filters.threshold_otsu(rgb_image[:, :, 2])
    tissue_RGB = np.logical_not(background_R & background_G & background_B)
    mask = tissue_S & (tissue_RGB)
    ret = np.array(mask).astype(np.uint8)
    return ret

def get_anno_list(anno_path, min_size, min_downsample):
    pts_list = []
    trees = etree.parse(anno_path).getroot()[0]

    for tree in trees:
        if (tree.get('PartOfGroup') == 'Tumor'):
            regions = tree.findall('Coordinates')
            for region in regions:
                coordinates = region.findall('Coordinate')
                pts = list()
                for coord in coordinates:
                      x = float(coord.get('X'))
                      y = float(coord.get('Y'))
                      x = np.clip(round(x/min_downsample), 0, round(min_size[0]))
                      y = np.clip(round(y/min_downsample), 0, round(min_size[1]))
                      pts.append((x,y))
                pts_list.append(pts)
    return pts_list

def get_anno_mask(pts_list, min_size):
    # 주의 numpy로 mask 생성 시 w,h 순서가 아닌 h,w !!
    mask = np.zeros((min_size[1],min_size[0])).astype(np.uint8)
    for pts in pts_list:
        point = [np.array(pts, dtype=np.int32)]
        mask = cv2.fillPoly(mask, point, 1)
    return mask

In [3]:
import os

base_slide_path = '/data3/yumi/TCGA_STAD'

# tif_files = [file for file in os.listdir(base_slide_path) if file.endswith('.tif')]
# tif_files_sorted = sorted(tif_files)

def find_svs_files(base_path):
    svs_files = []
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.endswith('.svs'):
                full_path = os.path.join(root, file)
                svs_files.append([file, full_path])
    return svs_files

svs_files = find_svs_files(base_slide_path)
svs_files_sorted = sorted(svs_files)

In [5]:
target_patch_size = 256
target_mpp = 1

tissue_ratio = 0.3
base_save_path = '/data3/yein/TCGA_STAD_patches'

for svs_file in svs_files_sorted:
    try:
        slide = openslide.open_slide(svs_file[1])
    
    except openslide.OpenSlideUnsupportedFormatError:
        print(f"Unsupported or missing image file: {svs_file[1]}")
        continue
    except Exception as e:
        print(f"Error opening slide: {e}")
        continue

    slide_name = os.path.basename(svs_file[0]).replace('.svs', '')
    save_path = os.path.join(base_save_path, slide_name)
    os.makedirs(save_path, exist_ok=True)

    min_level = slide.level_count - 1
    min_downsample = slide.level_downsamples[min_level]
    min_size = slide.level_dimensions[min_level]

    whole_region = slide.read_region(location=(0, 0), level=min_level, size=min_size)
    whole_image = np.array(whole_region)[..., :3]
    tissue_mask = get_tissue_mask(whole_image)

    level0_mpp = round(float(slide.properties.get('openslide.mpp-x')), 2)
    level0_patch_size = int((target_patch_size * target_mpp) / level0_mpp)
    level0_size = slide.level_dimensions[0]

    levelm_mpp = level0_mpp * min_downsample
    levelm_patch_size = int((target_patch_size * target_mpp) / levelm_mpp)

    patch_count = 0

    for start_level0_y in trange(0, level0_size[1], level0_patch_size):
        for start_level0_x in range(0, level0_size[0], level0_patch_size):
            start_levelm_x = int(start_level0_x / min_downsample)
            start_levelm_y = int(start_level0_y / min_downsample)
            end_levelm_x = int((start_level0_x + level0_patch_size) / min_downsample)
            end_levelm_y = int((start_level0_y + level0_patch_size) / min_downsample)
    
            tissue_mask_patch = tissue_mask[start_levelm_y:end_levelm_y, start_levelm_x:end_levelm_x]
    
            if get_ratio_mask(tissue_mask_patch) >= tissue_ratio:
                image_patch = np.array(slide.read_region(
                    location=(start_level0_x, start_level0_y),
                    level=0,
                    size=(level0_patch_size, level0_patch_size)
                )).astype(np.uint8)[..., :3]
                patch_count += 1
                execute_patch(image_patch, target_patch_size, save_path, start_level0_x, start_level0_y, patch_count, 0)  # 0 for non-tumor        

    # except Exception as e:
    #     print(f"Error processing file {svs_file}: {e}")

Error opening slide: cannot identify image file '/data3/yumi/TCGA_STAD/5dca3a38-9021-4a76-87ec-dfbe2a0a008b/TCGA-3M-AB46-01Z-00-DX1.70F638A0-BDCB-4BDE-BBFE-6D78A1A08C5B.svs'


100%|██████████| 79/79 [02:29<00:00,  1.89s/it]


Error opening slide: cannot identify image file '/data3/yumi/TCGA_STAD/59e9d91b-f7a2-4fc7-a53f-c47031f338e6/TCGA-B7-5816-01Z-00-DX1.1B05F96A-D5E2-4366-A098-A861313F3461.svs'
Error opening slide: cannot identify image file '/data3/yumi/TCGA_STAD/d27cc934-a03c-4055-afda-fa7283b827bf/TCGA-BR-6452-01Z-00-DX1.aec8246e-9dce-436d-9faa-df8fa1f6e47c.svs'


100%|██████████| 44/44 [02:38<00:00,  3.60s/it]


Error opening slide: cannot identify image file '/data3/yumi/TCGA_STAD/d1d64258-a7b1-474f-8e82-43025a6db293/TCGA-BR-6454-01Z-00-DX1.63fb6280-2b43-49b2-960a-eb9c0e8b617c.svs'
Error opening slide: cannot identify image file '/data3/yumi/TCGA_STAD/8450f85f-ab36-4a75-8169-e684cc0f7ab8/TCGA-BR-6455-01Z-00-DX1.402682fe-98b4-4381-85da-98510db17af9.svs'
Error opening slide: cannot identify image file '/data3/yumi/TCGA_STAD/8754286b-452e-4d96-95e8-848f39d7cc11/TCGA-BR-6456-01Z-00-DX1.2cb7b656-95e1-4f34-99e6-77a051fb79e5.svs'


...

Error opening slide: cannot identify image file '/data3/yumi/TCGA_STAD/81bb849e-48d2-4805-95f0-f18dc213b3cf/TCGA-VQ-AA6B-01Z-00-DX1.C6CB7290-4A83-4963-8840-68BC208D1232.svs'


100%|██████████| 71/71 [02:16<00:00,  1.92s/it]


Error opening slide: cannot identify image file '/data3/yumi/TCGA_STAD/a615ab15-077e-417c-9171-5aebda2b7261/TCGA-VQ-AA6F-01Z-00-DX1.EA90F7CF-FE3F-4FED-A8E0-C83FEE96F4CA.svs'
Error opening slide: cannot identify image file '/data3/yumi/TCGA_STAD/ec1703a8-ada8-40f8-a578-6af2c51317aa/TCGA-VQ-AA6G-01Z-00-DX1.3BDB7578-931A-47DB-933F-C7CA7CD50EDE.svs'


100%|██████████| 88/88 [03:04<00:00,  2.10s/it]

Error opening slide: cannot identify image file '/data3/yumi/TCGA_STAD/4cebb542-7b44-44bb-b048-4414d88410c0/TCGA-VQ-AA6J-01Z-00-DX1.212AF324-26E9-4807-A64B-6325D6E94507.svs'
Error opening slide: cannot identify image file '/data3/yumi/TCGA_STAD/c09810f3-d4d9-4e2f-a87f-1604d848edfc/TCGA-VQ-AA6K-01Z-00-DX1.8AF14797-0C31-4DF3-94A4-868EC96E7BB1.svs'
Error opening slide: cannot identify image file '/data3/yumi/TCGA_STAD/cba52a6e-ef93-424c-b508-d9fc625e0d65/TCGA-ZA-A8F6-01Z-00-DX1.89510833-5BC8-4983-817E-0E9244824168.svs'
Error opening slide: cannot identify image file '/data3/yumi/TCGA_STAD/e895d806-6ee8-4069-825c-b9e7494071da/TCGA-ZQ-A9CR-01Z-00-DX1.C9AC9991-7054-4068-A7AA-9D1EB7F040A3.svs'


In [ ]:
# target_patch_size = 224
# target_mpp = 1000

# tissue_ratio = 0.3
# base_save_path = './path'
# base_slide_path = './path'

# for slide_filename in tif_files_sorted:
#     slide_path = os.path.join(base_slide_path, slide_filename)
#     save_path = os.path.join(base_save_path, slide_filename.split('.')[0])
#     os.makedirs(save_path, exist_ok=True)
    
#     slide = openslide.open_slide(slide_path)
    
#     min_level = slide.level_count - 1
#     min_downsample = slide.level_downsamples[slide.level_count - 1]
#     min_size = slide.level_dimensions[slide.level_count - 1]
    
#     whole_region = slide.read_region(location=(0, 0), level=min_level, size=min_size)
#     whole_image = np.array(whole_region)[..., :3]
#     tissue_mask = get_tissue_mask(whole_image)
    
#     level0_mpp = round(float(slide.properties.get('openslide.mpp-x')), 2)
#     level0_patch_size = int((target_patch_size * target_mpp) / level0_mpp)
#     level0_size = slide.level_dimensions[0]

#     levelm_mpp = level0_mpp * min_downsample
#     levelm_patch_size = int((target_patch_size * target_mpp) / levelm_mpp)
    
#     patch_count = 0
    
#     for start_level0_y in trange(0, level0_size[1], level0_patch_size):
#         for start_level0_x in range(0, level0_size[0], level0_patch_size):
#             start_levelm_x = int(start_level0_x / min_downsample)
#             start_levelm_y = int(start_level0_y / min_downsample)
#             end_levelm_x = int((start_level0_x + level0_patch_size) / min_downsample)
#             end_levelm_y = int((start_level0_y + level0_patch_size) / min_downsample)
    
#             tissue_mask_patch = tissue_mask[start_levelm_y:end_levelm_y, start_levelm_x:end_levelm_x]
    
#             if get_ratio_mask(tissue_mask_patch) >= tissue_ratio:
#                 image_patch = np.array(slide.read_region(
#                     location=(start_level0_x, start_level0_y),
#                     level=0,
#                     size=(level0_patch_size, level0_patch_size)
#                 )).astype(np.uint8)[..., :3]
#                 patch_count += 1
#                 execute_patch(image_patch, target_patch_size, save_path, start_level0_x, start_level0_y, patch_count, 0)  # 0 for non-tumor